In [209]:
import numpy as np
import torch

from torch.utils.data import Dataset, DataLoader


1. generate points. Just a line so a vector with 10 points with varying starting points that is sorted. return the first point. It is vector, label

Encapsulate that in a Dataloader object.


In [210]:
amount_of_points = 10
# min_val = 0
max_val = 30

def get_line():
    min_val = np.random.randint(low=0, high= max_val//2)
    line_points = np.random.randint(low=min_val, high=max_val, size=(amount_of_points,))
    line_points.sort()
    torch_line_points = torch.from_numpy(line_points)
    return torch_line_points, torch_line_points.sum()

print(get_line())
print(type(get_line()[1]))

class CustomDataset(Dataset):
    def __init__(self, size):
        self.size = size
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, index):
        input_data, label = get_line()
        return input_data, label
    
dataset_size = 10000
batch_size = 16


train_dataset = CustomDataset(dataset_size)
test_dataset = CustomDataset(dataset_size//100)

# No need for shuffle
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


(tensor([10, 11, 11, 13, 14, 14, 14, 22, 27, 28]), tensor(164))
<class 'torch.Tensor'>


2. Embedding layer that takes the points and embedds them into a different space.

In [211]:
# TODO what about the embedding dim
embedding_dim = 64
embedding_layer = torch.nn.Embedding(num_embeddings=max_val + 1, embedding_dim=embedding_dim)

3. Positional encoding: ignore for now

In [212]:
# ignored

4. Attention Layer. Also a network/Layer

In [213]:
from torch import nn
import torch
import torch.nn.functional as F

class AttentionLayer(nn.Module):
    def __init__(self, feature_size):
        super(AttentionLayer, self).__init__()
        self.feature_size = feature_size

        # calculate Q, K, V from the same source
        self.k = nn.Linear(feature_size, feature_size)
        self.q = nn.Linear(feature_size, feature_size)
        self.v = nn.Linear(feature_size, feature_size)

    def forward(self, x, mask=None):
        # linear trans
        keys = self.k(x)
        queries = self.q(x)
        values = self.v(x)

        # move into init?
        # Why the f do I need the first parameter as a tensor? Wrong function call? how about np.sqrt?
        scaling_factor = torch.sqrt(torch.tensor(self.feature_size, dtype=torch.float32))
        
        scores = torch.matmul(queries, keys.transpose(-2, -1)) / scaling_factor

        # optional: add mask here

        # appy softmax, convert scores to floating point otherwise the softmax doesn't work
        # TODO figure out why
        attention_weights = F.softmax(scores.float(), dim=1)

        output = torch.matmul(attention_weights, values)

        return output

Final network, with all layers + a linear layer (should suffice)

In [214]:
class Transfomer(nn.Module):
    def __init__(self):
        super(Transfomer, self).__init__()
        self.embedding = torch.nn.Embedding(num_embeddings=max_val + 1, embedding_dim=embedding_dim)
        self.attention_layer = AttentionLayer(feature_size=embedding_dim)
        self.lin_net1 = nn.Linear(embedding_dim, out_features=64)
        self.lin_net2 = nn.Linear(64, out_features=1)
        self.lin_net3 = nn.Linear(10, 1)

    def forward(self, x):
        x = self.embedding(x)
        x = self.attention_layer(x)
        x = self.lin_net1(x)
        x = self.lin_net2(x)
        x = x.squeeze()
        x = self.lin_net3(x)
        x = x.squeeze()
        return x

5. Training loop

In [215]:
def onehotencodevalue(size:int, label:int):
    vector = torch.zeros(size)
    vector[label] = 1.0

    return vector

def onehotencodebatchvalues(size:int, labels):
    oneHotencodedBatch = torch.zeros((len(labels), size))

    for idx, label in enumerate(labels):
        OHE_vector = onehotencodevalue(size, label)
        oneHotencodedBatch[idx] = OHE_vector
    
    return oneHotencodedBatch

OHEBathc = onehotencodebatchvalues(max_val + 1, [4, 7, 9, 10])
OHEBathc.shape

torch.Size([4, 31])

In [216]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and  dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch_number, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        # take the highest predicted class.
        # TODO THIS SEEMS WRONG; TAKING THE HIGHES PREDICTED CLASS WILL STIFEN LEARNING NO?
        pred = model(X)

        loss = loss_fn(pred.float(), y.float())

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch_number % 100 == 0:
            loss, current = loss.item(), (batch_number + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

6. Testing loop

In [217]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred.float(), y).item()
            correct += (pred == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

7. Execution 

In [218]:
model = Transfomer()
learning_rate = 0.1

# defines the "correct" the output is
loss_fn = torch.nn.HuberLoss(reduction='mean', delta=0.02)
# defines how to adapt the model parameters depending on the (input, loss)
# here we use stochastic gradient descent
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
	# testing for each epoch to track the models performance during training.
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 3.607677  [   16/10000]
loss: 3.621504  [ 1616/10000]
loss: 0.348752  [ 3216/10000]
loss: 0.164937  [ 4816/10000]
loss: 0.529379  [ 6416/10000]
loss: 0.142390  [ 8016/10000]
loss: 0.077591  [ 9616/10000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.223382 

Epoch 2
-------------------------------
loss: 0.230664  [   16/10000]
loss: 0.704675  [ 1616/10000]
loss: 0.660638  [ 3216/10000]
loss: 0.205202  [ 4816/10000]
loss: 0.144659  [ 6416/10000]
loss: 0.419126  [ 8016/10000]
loss: 0.107252  [ 9616/10000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.368977 

Epoch 3
-------------------------------
loss: 0.345632  [   16/10000]
loss: 0.177451  [ 1616/10000]
loss: 0.557034  [ 3216/10000]
loss: 0.072028  [ 4816/10000]
loss: 0.074974  [ 6416/10000]
loss: 0.524614  [ 8016/10000]
loss: 0.310789  [ 9616/10000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.448795 

Epoch 4
-------------------------------
loss: 0.475042  [   16/10000]
loss: 0.146410  [ 1616/1000

In [222]:
import math

x,y = get_line()
model_output = model(x)
print("model input: ", x)
print("expected output: first element: ", y.item())
print("actual output: ", model_output.item())

difference = abs(model_output - y)
print("difference: ", difference)

if difference < 1:
    print("not bad")
else:
    print("needs some finetuning still")

model input:  tensor([17, 17, 20, 20, 20, 22, 24, 24, 26, 28])
expected output: first element:  218
actual output:  217.1650848388672
difference:  tensor(0.8349, grad_fn=<AbsBackward0>)
not bad
